In [121]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
%matplotlib inline

CWD = "/".join(os.getcwd().split("/")[:-2])

In [132]:
df = pd.read_csv(f"{CWD}/analysis/results/mcmc/mcmcoutput.csv")
df = df.loc[[0,2,4]]

In [133]:
def add_val_with_percentiles(df, val, out):
    suff = ["_p16","_p50","_p84"]
    for s in suff:
        df[val+s] = df[val+s].apply(lambda x: float(x))
    
    df["vplus"] = df.apply(lambda x: x[val+suff[2]]-x[val+suff[1]], axis=1)
    df["vminus"] = df.apply(lambda x: x[val+suff[1]]-x[val+suff[0]], axis=1)
    print(df[val+suff[1]])
    #vminus = df[val+suff[2]]-df[val+suff[1]]
    #df[r"$t_0$"] = df.apply(lambda x: fr"${x.t0_p50}(^{float(x.t0_p84)-float(x.t0_p50)}_{float(x.t0_p50)-float(x.t0_p16)})$", axis=1)
    power = int(np.round(np.log10(float(np.min(df.vplus)))))
    print(power, "POWER")
    powerabs = int(np.round(np.log10(float(np.min( df[val+suff[1]])))))
    print(powerabs, "POWERABS")
    if abs(powerabs) > 5:
        out = out + f"$\cdot 10^{powerabs}"
        out = out.replace("^","^{")
        out = out + "}$"
        df.vminus = df.vminus/(10**(power))
        df.vplus = df.vplus/(10**(power))
        df[val+suff[1]] = df[val+suff[1]]/(10**powerabs)
        
        df[out] = df.apply(lambda x: f"{x[val+suff[1]]:.{powerabs-power}f}(^{x.vplus:.0f}_{x.vminus:.0f})", axis=1)
        df[out] = df[out].apply(lambda x: x.replace("^","^{").replace("_","}_{").replace(")","})"))
    else:
        df[out] = df.apply(lambda x: f"{x[val+suff[1]]:.{abs(powerabs-power)}f}(^{x.vplus/(10**power):.0f}_{x.vminus/(10**power):.0f})", axis=1)
        df[out] = df[out].apply(lambda x: x.replace("^","^{").replace("_","}_{").replace(")","})"))
    return df
                                
valout = [("t0", "$t_0$ (BJD)"),
          ("Eflare","$E_{flare}$ (erg)"),
          ("ED_distr", "$ED$ (s)")]
for val, out in valout:
    df = add_val_with_percentiles(df, val, out)
df                                    

0    1358.512647
2    1331.664065
4    1331.824582
Name: t0_p50, dtype: float64
-5 POWER
3 POWERABS
0    5.075236e+33
2    6.886252e+34
4    4.833674e+34
Name: Eflare_p50, dtype: float64
32 POWER
34 POWERABS
0    45333.419214
2     7762.928334
4     5449.039645
Name: ED_distr_p50, dtype: float64
2 POWER
4 POWERABS


,date,ID,burnin,steps,walkers,nparam,t0_p16,t0_p50,t0_p84,theta_a_p16,...,ED_distr_p50,ED_distr_p84,Eflare_p16,Eflare_p50,Eflare_p84,vplus,vminus,$t_0$ (BJD),$E_{flare}$ (erg)$\cdot 10^{34}$,$ED$ (s)
0,13_12_2019_10_18,100004076,3000,5000,32,6,1358.512411,1358.512647,1358.512939,71.6583396311642,...,45333.419214,47021.176773,4.709872e+33,0.507524,5.264186e+33,1687.757559,3263.530055,1358.51265(^{29}_{24}),0.51(^{2}_{4}),45333.42(^{17}_{33})
2,13_01_2020_11_28,237880881a,10000,11824,32,10,1331.664045,1331.664065,1331.664086,25.3737134205418,...,7762.928334,9356.479936,5.422910e+34,6.88625,8.299843e+34,1593.551602,1649.637934,1331.66407(^{2}_{2}),6.89(^{141}_{146}),7762.93(^{16}_{16})
4,13_01_2020_11_28,237880881b,10000,11824,32,10,1331.824402,1331.824582,1331.824865,43.693619297167,...,5449.039645,5712.438027,4.526045e+34,4.83367,5.067326e+34,263.398383,346.792645,1331.82458(^{28}_{18}),4.83(^{23}_{31}),5449.04(^{3}_{3})


In [104]:
# mission ID, other ID, QCS, SpT, RA, Dec, vsini, Prot_d
#df["mission ID"] = df.apply(lambda x: f"{x.prefix} {x.ID}", axis=1)
df[r"$t_0$"] = df.apply(lambda x: f"{x.BJDoff + x.tstart:.3f}", axis=1)
df[r"ED"] = df.apply(lambda x: f"{x.ED}({x.e_ED})",axis=1)
df[r"$v\sin i$"] = df.apply(lambda x: f"{x.vsini_kms}({x.e_vsini_kms})",axis=1)

AttributeError: ("'Series' object has no attribute 'BJDoff'", 'occurred at index 0')

In [115]:
df1 = df.rename(index=str, columns={"identifier" : "other ID",
                                    "Gaia_G" : r"$G_\mathrm{Gaia}$",
                                    "Prot_d": r"$P_\mathrm{rot}$",
                                    "cadence_min":"cadence",
                                    "J":r"$J$"
                                   })

In [116]:
cols = ["ID", "QCS", "other ID", "RA", "Dec", r"$J$",
        "SpT", "cadence", r"$t_0$", r"$P_\mathrm{rot}$",
        r"$v\sin i$", "ED"]
units =  ["","","","","deg","deg","mag","min","BJD","d","km/s","s"]
df2 = df1[cols]
df3 = df2.append(dict(zip(cols,units)), ignore_index=True)
df3 = df3.sort_index(axis=0,ascending=False).reset_index()
df3 = df3[cols]
df3

,ID,QCS,other ID,RA,Dec,$J$,SpT,cadence,$t_0$,$P_\mathrm{rot}$,$v\sin i$,ED
0,,,,,deg,deg,mag,min,BJD,d,km/s,s
1,TIC 277539431,12,WISEA J105515.71-735611.3,163.815,-73.9364,10.63,M7,2,2458641.835,0.19,nan(nan),4495(394)
2,TIC 44984200,10,SCR J0838-5855,129.508,-58.9331,10.309,M6,2,2458588.030,0.113,nan(nan),3809(135)
3,TIC 237880881,1,2MASS J01180670-6258591,19.528,-62.9831,11.53,M5,2,2458331.700,0.35125,14.4(2.6),6020(298)
4,EPIC 212035340,18,MASS J08371832+2050349,129.326,20.843,15.9,M8,30,2458270.750,0.193,30.0(12.0),1954494(483655)
5,KIC 100004076,14,WISEP J190648.47+401106.8,286.7,40.1857,13.078,L1,1,2456191.550,0.37015,11.2(2.2),10414(886)


In [117]:
references = [("a","gizis2013",[]),
              ("b", "paudel2019",[]),
              ("c","kraus2014",["14.4(2.6)"]),
             ]

In [118]:
nc = 'c' * (df3.shape[1]-2) #number of middle columns
footnote_pref= "\multicolumn{" + str(df3.shape[1]-2) + "}{l}" 
footnote_suf = "\n"
fs = []
for alpha, ref, repl in references:
    f1 = footnote_pref + "{$^" + alpha + "$}{\citet\{" + ref + "}}" + footnote_suf
    fs.append(f1)

stri = df3.to_latex(index=False,escape=False, column_format=f"l{nc}r")
stri = stri.replace("\midrule","\hline")

for alpha, ref, repl in references:
    if len(repl) > 0:
        for r in repl:
            stri = stri.replace(r,r + r"$^" + alpha + "$")

repl = " ".join(fs)
stri = stri.replace("\\bottomrule","\hline\n" + repl)
print(stri)

\begin{tabular}{lccccccccccr}
\toprule
             ID & QCS &                   other ID &       RA &      Dec &     $J$ &  SpT & cadence &        $t_0$ & $P_\mathrm{rot}$ &   $v\sin i$ &               ED \\
\hline
                &     &                            &          &      deg &     deg &  mag &     min &          BJD &                d &        km/s &                s \\
  TIC 277539431 &  12 &  WISEA J105515.71-735611.3 &  163.815 & -73.9364 &   10.63 &   M7 &       2 &  2458641.835 &             0.19 &    nan(nan) &        4495(394) \\
   TIC 44984200 &  10 &             SCR J0838-5855 &  129.508 & -58.9331 &  10.309 &   M6 &       2 &  2458588.030 &            0.113 &    nan(nan) &        3809(135) \\
  TIC 237880881 &   1 &    2MASS J01180670-6258591 &   19.528 & -62.9831 &   11.53 &   M5 &       2 &  2458331.700 &          0.35125 &   14.4(2.6)$^c$ &        6020(298) \\
 EPIC 212035340 &  18 &    MASS J08371832+2050349  &  129.326 &   20.843 &    15.9 &   M8 &      30 

In [119]:
## Add references

In [120]:
with open(f"{CWD}/paper/draft/tables/lcs.tex", "w") as f:
    f.write(stri)